<h3> Web Scrapping all the attributes

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
# from time import sleep
# from random import random
#create a list for all whisklies' name
url = "https://distiller.com/search?page=1"
names = []
links = []
whiskey_info = dict()
for i in range(1,208):
    url = "https://distiller.com/search" + "?page=" + str(i)
    response = requests.get(url)
    response = BeautifulSoup(response.content, 'lxml')
    for i in response.find_all('li',{'class':"spirit whiskey-content"}):
        name = i.find('div', {'class':'name'}).get_text()
        names.append(name)
        link = i.find('a').get('href')
        list_ = list(get_info("https://distiller.com"+link))
        links.append("https://distiller.com"+link)
        whiskey_info[name]=list_

In [ ]:
def get_info(link):
    response = requests.get(link)
    results_page = BeautifulSoup(response.content,'lxml')
    try:
        type_ = results_page.find("h2", class_ = "ultra-mini-headline type").get_text()
    except:
        type_ = "NA"
    origin = results_page.find("h2", class_ = "ultra-mini-headline location middleweight").get_text()
    import re
    pattern = r'([\w,-. ]+) // ([\w,-. ]+)'
    try:
        origin = re.search(pattern, origin).group(2) 
    except:
        origin = "NA"
    try:
        distillerscore = results_page.find("span", class_ = "expert-rating").get_text().strip('\n')
    except:
        distillerscore = "NA"
    try:
        averagerating = results_page.find("span", itemprop = "ratingValue").get_text().strip('\n')
    except:
        averagerating = 0
    age_result = results_page.find("li", class_ = "detail age").get_text()
    try:
        age = re.search(r'([\w ]+)\n([\w ]+)', age_result).group(2)
    except:
        age = 'NA'
    abv_result = results_page.find("li", class_="detail abv").get_text()
    try:
        abv_score = re.search(r'([\w ]+)\n([\w. ]+)',abv_result).group(2)
    except:
        abv_score = 0
    style_result = results_page.find("li", class_ = "detail whiskey-style").get_text()
    try:
        whiskey_style = re.search(r'([\w\S. ]+)\n([\w.% ]+)',style_result).group(1)
    except:
        whiskey_style = "NA"
    cask_result = results_page.find("li", class_="detail cask-type").get_text()
    try:
        cask_type = re.search(r'([\w\S. ]+)\n([\w\S ]+)',cask_result).group(2)
    except:
        cask_type = "NA"
    try:
        flavor_headlines = results_page.find("h3", class_="secondary-headline flavors middleweight").get_text()
    except: 
        flavor_headlines = "NA"
    try:
        import ast
        flavor_content = results_page.find("canvas", class_= "js-flavor-profile-chart")["data-flavors"]
        flavor_content_dict = ast.literal_eval(flavor_content)
    except:
        flavor_content_dict = "NA"
    try:
        tastes_note = results_page.find("p", itemprop="reviewBody").get_text().strip('\r\n"')
    except:
        tastes_note = "NA"
    try:
        description = results_page.find("p", class_ = "description").get_text().strip('\r\n')
    except:
        description = "NA"
    try:
        taste_count = results_page.find("span", class_ = "count").get_text()
    except:
        taste_count = 0
    try:
        costs = results_page.find_all('li',class_ = "stat cost")
        for cost in costs:
            result = cost.find('div',{"data-behavior":"cost-value"})
        cost_count = str(result).split(">")[0].split("=")[1].split('-')[1][0]

    except:
        cost_count = 0
    return type_, origin,distillerscore,averagerating,age,abv_score,whiskey_style,cask_type,flavor_headlines,flavor_content_dict,tastes_note,description,taste_count,cost_count


In [ ]:
import pandas as pd
info_total = pd.DataFrame({'whiskey_name': 0, 'type':0, 'origin': 0, 'distillerscore':0, 'averagerating':0, 'age':0, 'abv_score':0,'whiskey_style':0, 'cask_type':0, 'flavor_headlines':0,'smoky':0,'peaty':0,'spicy':0,'herbal':0,'oily':0,'full_bodied':0,'rich':0,'sweet':0, 'briny':0,'salty':0,'vanilla':0,'tart':0,'fruity':0,'floral':0, 'tastes_note':0,'description':0,'taste_count':0,'cost_count':0}, index=[0])
for key,value in whiskey_info.items():
    flavor_dict=value[9]
    if flavor_dict=="NA":
        info_temp = pd.DataFrame({'whiskey_name': key, 'type':value[0], 'origin': value[1], 'distillerscore':value[2], 'averagerating':value[3], 'age':value[4], 'abv_score':value[5],'whiskey_style':value[6], 'cask_type':value[7], 'flavor_headlines':value[8],'smoky':0,'peaty':0,'spicy':0,'herbal':0,'oily':0,'full_bodied':0,'rich':0,'sweet':0, 'briny':0,'salty':0,'vanilla':0,'tart':0,'fruity':0,'floral':0, 'tastes_note':value[10],'description':value[11],'taste_count':value[12],'cost_count':value[-1]},index=[0])
    else:
        info_temp = pd.DataFrame({'whiskey_name': key, 'type':value[0], 'origin': value[1], 'distillerscore':value[2], 'averagerating':value[3], 'age':value[4], 'abv_score':value[5],'whiskey_style':value[6], 'cask_type':value[7], 'flavor_headlines':value[8],'smoky':flavor_dict['smoky'],'peaty':flavor_dict['peaty'],'spicy':flavor_dict['spicy'],'herbal':flavor_dict['herbal'],'oily':flavor_dict['oily'],'full_bodied':flavor_dict['full_bodied'],'rich':flavor_dict['rich'],'sweet':flavor_dict['sweet'], 'briny':flavor_dict['briny'],'salty':flavor_dict['salty'],'vanilla':flavor_dict['vanilla'],'tart':flavor_dict['tart'],'fruity':flavor_dict['fruity'],'floral':flavor_dict['floral'], 'tastes_note':value[10],'description':value[11],'taste_count':value[12],'cost_count':value[-1]},index=[0])
    
    info_total = pd.concat([info_total, info_temp])
    info_total.reset_index(inplace=True,drop=True)
info_total=info_total[1:]

In [ ]:
info_total.to_csv("whiskey_data_new.csv")

<h3> Web Scrapping all the comments

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests

In [ ]:
import requests
from bs4 import BeautifulSoup
base_url = "https://distiller.com"
url = base_url + "/" + "search"
response = requests.get(url)
response = BeautifulSoup(response.content, 'lxml')
whiskey_name1, whiskey_name2, whiskey_link1, whiskey_link2 = (list() for _ in range(4))
whiskey_link1 = list()
for i in response.find_all('li',{'class':"spirit whiskey-content"}):
        whiskey_name1.append(i.find('div', {'class':'name'}).get_text())
        whiskey_link1.append(i.find('a').get('href'))
for i in range(2,218):
    print(i)
    url = "https://distiller.com/search" + "?page=" + str(i)
    response = requests.get(url)
    response = BeautifulSoup(response.content, 'lxml')
    for i in response.find_all('li',{'class':"spirit whiskey-content"}):
        whiskey_name2.append(i.find('div', {'class':'name'}).get_text())
        whiskey_link2.append(i.find('a').get('href'))
        
whiskey_name = whiskey_name1 + whiskey_name2
whiskey_link = whiskey_link1 + whiskey_link2

import pickle
with open('whiskey_name', 'wb') as fp:
    pickle.dump(whiskey_name, fp)
with open ('whiskey_name', 'rb') as fp:
    itemlist = pickle.load(fp)
with open('whiskey_link', 'wb') as fp:
    pickle.dump(whiskey_link, fp)
# create a dictionary of whiskey with name:taste link as pairs
whiskey_dict = dict(zip(whiskey_name, whiskey_link))

import pandas as pd
import numpy as np
import math
whiskey_dict = dict(zip(whiskey_name, whiskey_link))
base_url = 'https://distiller.com'
df_total = pd.DataFrame({'whiskey_name': 0, 'name':0, 'rating': 0, 'comment':0, 'time':0}, index=[0])
for indx, w_name in enumerate(whiskey_name[1365:1465]):
    print(indx, w_name)
    url_whiskey =  base_url + whiskey_dict[w_name]
    print(url_whiskey)
    response = requests.get(url_whiskey)
    response = BeautifulSoup(response.content, 'lxml')
    taste_link = response.find('li',{'class':'tab-tastes-header'}).find('div',{'class':'tab-content'}).find('a').get('href')
    url_taste = base_url + taste_link
    name1, rating1, comment1, time1 = (list() for _ in range(4))
    response = requests.get(url_taste)
    response = BeautifulSoup(response.content, 'lxml')
    count = int(response.find('span',{'class':'count'}).get_text())
    last_page = math.ceil(count/10)
    name1 =[i.get_text().strip() for i in response.find_all('h3',{'class':'mini-headline name username truncate-line'})]
    for i in response.find_all('div',{'class':'high-level-info'}):
        if i.find('div',{'class':'rating-display'}):
            rating1.append(float(i.find('div',{'class':'rating-display__value'}).get_text().strip()))
        else:
            rating1.append('None')
    for i in response.find_all('li',{'class':'taste spirit-taste spirit-list-item'}):
        if i.find('div',{'class':'body'}): 
            comment1.append(i.find('div',{'class':'body'}).get_text().strip())
        else:
            comment1.append('None')
    time1 = [i.get('datetime') for i in response.find_all('time',{'class':'timeago'})]
    name2, rating2, comment2, time2 = (list() for _ in range(4))
    for i in range(2,last_page+1):
        url = url_taste + "?page=" + str(i)
        response = requests.get(url)
        response = BeautifulSoup(response.content, 'lxml')
        name2 +=[j.get_text().strip() for j in response.find_all('h3',{'class':'mini-headline name username truncate-line'})]
        for k in response.find_all('div',{'class':'high-level-info'}):
            if k.find('div',{'class':'rating-display'}):
                rating2.append(float(k.find('div',{'class':'rating-display__value'}).get_text().strip()))
            else:
                rating2.append('None')
        for j in response.find_all('li',{'class':'taste spirit-taste spirit-list-item'}):
            if j.find('div',{'class':'body'}):
                comment2.append(j.find('div',{'class':'body'}).get_text().strip())
            else:
                comment2.append('None')
        time2 += [j.get('datetime') for j in response.find_all('time',{'class':'timeago'})]
    name = name1 + name2
    rating = rating1 + rating2
    comment = comment1 + comment2
    time = time1 + time2
    df_temp = pd.DataFrame({'whiskey_name': w_name, 'name':name, 'rating': rating, 'comment':comment, 'time':time})
    df_total = pd.concat([df_total, df_temp])
    df_total.reset_index(inplace=True,drop=True)
df_total = df_total[1:]
df_total.to_csv('df_master.csv', encoding ='utf-8')

In [ ]:
import matplotlib.pyplot as plt
avg_rating.plot.bar()
plt.show()